In [ ]:
# utils_robot.py
# 启动并连接机械臂，导入各种工具包

import time
import math
import smbus2 as smbus   #import smbus 树莓派自己修改回来
import cv2
import numpy as np
from utils_pump import *


#     prescale = math.floor(prescaleval + 0.5)

#     oldmode = self.bus.read_byte_data(self.address, 0x00)
#     newmode = (oldmode & 0x7F) | 0x10
#     self.write(0x00, newmode)
#     self.write(0xFE, int(prescale))import RPi.GPIO as GPIO

# 初始化PCA9685模块     这个类用于与PCA9685模块进行通信。
# print('初始化PCA9685模块')
# class PCA9685:

#     def __init__(self, address=0x40, debug=False):
#         self.bus = smbus.SMBus(1)  # I2C总线1
#         self.address = address
#         self.debug = debug
#         self.write(0x00, 0x00)  # 初始化PCA9685
  
#     def write(self, reg, value):                         #写入寄存器的 write() 方法
#         self.bus.write_byte_data(self.address, reg, value)
    
#     def setPWMFreq(self, freq):                            #setPWMFreq() 用于设置PCA9685的PWM输出频率
#     prescaleval = 25000000.0
#     prescaleval /= 4096.0
#     prescaleval /= float(freq)
#     prescaleval -= 1.0
#     self.write(0x00, oldmode)
#     time.sleep(0.005)
#     self.write(0x00, oldmode | 0x80)

#         prescaleval = 25000000.0
#         prescaleval /= 4096.0
#         prescaleval /= float(freq)
#         prescaleval -= 1.0
#         prescale = math.floor(prescaleval + 0.5)

#         oldmode = self.bus.read_byte_data(self.address, 0x00)
#         newmode = (oldmode & 0x7F) | 0x10
#         self.write(0x00, newmode)
#         self.write(0xFE, int(prescale))
#         self.write(0x00, oldmode)
#         time.sleep(0.005)
#         self.write(0x00, oldmode | 0x80)

#     def setPWM(self, channel, on, off):         #def setPWM(self, channel, on, off):
#     self.write(0x06 + 4 * channel, on & 0xFF)
#     self.write(0x07 + 4 * channel, on >> 8)
#     self.write(0x08 + 4 * channel, off & 0xFF)
#     self.write(0x09 + 4 * channel, off >> 8)

#         self.write(0x06 + 4 * channel, on & 0xFF)
#         self.write(0x07 + 4 * channel, on >> 8)
#         self.write(0x08 + 4 * channel, off & 0xFF)
#         self.write(0x09 + 4 * channel, off >> 8)
    
#     def setServoPulse(self, channel, pulse):
#         pulse = pulse * 4096 / 20000  # 将脉冲转换为12位的值
#         self.setPWM(channel, 0, int(pulse))

pca = PCA9685(0x40, debug=False)
pca.setPWMFreq(50)  # 设置PWM频率为50Hz


# 控制舵机6的函数，已移除所有GPIO相关代码，完全依赖PCA9685的PWM信号来控制舵机6的机械爪抓取。
def control_gripper(action):
    if action == "open":
        pwm.setServoPulse(5, cal_pwm(90))  # 打开夹具（舵机6）
        print("夹具打开")
    elif action == "close":
        pwm.setServoPulse(5, cal_pwm(144))  # 抓取（舵机6）
        print("夹具抓取")
    else:
        print("无效的动作指令")

# 测试夹具控制
control_gripper("open")   # 打开夹具
time.sleep(2)
control_gripper("close")  # 抓取


def cal_pwm(degree):
    pwm = degree/270*2000+500
    return pwm






def set_servo_angle(channel, angle):
    # 根据不同通道的舵机设定脉宽范围和角度限制
    if channel == 0:  # 舵机1，0-180°，OUT0
        pulse = 500 + (angle / 180.0) * 2000
        pulse = max(500, min(2500, pulse))  # 限制脉宽范围在500-2500
    elif channel == 1:  # 舵机2，0-270°，OUT1
        pulse = 500 + (angle / 270.0) * 2000
        pulse = max(500, min(2500, pulse))  # 限制脉宽范围在500-2500
    elif channel == 2:  # 舵机3，0-270°，OUT2
        pulse = 550 + (angle / 270.0) * 1950
        pulse = max(550, min(2500, pulse))  # 限制脉宽范围在550-2500
    elif channel == 3:  # 舵机4，0-270°，OUT3
        pulse = 500 + (angle / 270.0) * 2000
        pulse = max(500, min(2500, pulse))  # 限制脉宽范围在500-2500
    elif channel == 4:  # 舵机5，0-180°，OUT4
        pulse = 500 + (angle / 180.0) * 2000
        pulse = max(500, min(2500, pulse))  # 限制脉宽范围在500-2500
    elif channel == 5:  # 舵机6，0-180°，OUT5
        pulse = 1500 + (angle / 180.0) * 600
        pulse = max(1500, min(2100, pulse))  # 限制脉宽范围在1500-2100
    else:
        print("无效的通道")
        return

    pca.setServoPulse(channel, pulse)  # 设置对应通道的PWM脉宽
    print(f"设置通道 {channel} 的舵机到角度 {angle}°，脉宽为 {pulse}")



# 舵机角度和PWM限制的配置   我新加的舵机运动物理限制
servo_limits = {
    0: {"min_angle": 0, "max_angle": 180, "min_pulse": 500, "max_pulse": 2500},   # 舵机1
    1: {"min_angle": 0, "max_angle": 270, "min_pulse": 850, "max_pulse": 2300},   # 舵机2
    2: {"min_angle": 0, "max_angle": 270, "min_pulse": 550, "max_pulse": 2500},   # 舵机3
    3: {"min_angle": 0, "max_angle": 270, "min_pulse": 500, "max_pulse": 2500},   # 舵机4
    4: {"min_angle": 0, "max_angle": 180, "min_pulse": 500, "max_pulse": 2500},   # 舵机5
    5: {"min_angle": 0, "max_angle": 180, "min_pulse": 1500, "max_pulse": 2100}   # 舵机6
}

def set_servo_angle(channel, angle):
    if channel not in servo_limits:
        print("无效的通道")
        return
    
    # 获取当前通道的舵机限制
    limits = servo_limits[channel]
    min_angle = limits["min_angle"]
    max_angle = limits["max_angle"]
    min_pulse = limits["min_pulse"]
    max_pulse = limits["max_pulse"]

    # 确保角度在允许范围内
    if angle < min_angle or angle > max_angle:
        print(f"角度超出范围！通道 {channel} 的有效角度范围是 {min_angle}° 到 {max_angle}°")
        return

    # 计算对应的PWM脉宽
    pulse = min_pulse + (angle - min_angle) * (max_pulse - min_pulse) / (max_angle - min_angle)
    pulse = max(min_pulse, min(max_pulse, pulse))  # 限制脉宽在允许范围内

    pca.setServoPulse(channel, pulse)  # 设置对应通道的PWM脉宽
    print(f"设置通道 {channel} 的舵机到角度 {angle}°，脉宽为 {pulse}")





def back_zero():
    '''
    机械臂归零到指定的PWM脉宽
    '''
    print('机械臂归零')
    pwm_values = [1500, 1500, 1500, 1500, 1610, 1500]  # 指定的PWM脉宽值

    for i, pwm in enumerate(pwm_values):
        pca.setServoPulse(i, pwm)  # 设置对应通道的PWM脉宽
        print(f'设置通道 {i} 的PWM脉宽为 {pwm}')

    time.sleep(3)  # 停顿3秒，确保机械臂完成动作





def relax_arms():
    '''
    放松机械臂关节到指定的PWM脉宽
    '''
    print('放松机械臂关节')
    pwm_values = [1500, 1200, 550, 1200, 1610, 1400]  # 指定的PWM脉宽值

    for i, pwm in enumerate(pwm_values):
        pca.setServoPulse(i, pwm)  # 设置对应通道的PWM脉宽
        print(f'设置通道 {i} 的PWM脉宽为 {pwm}')

    time.sleep(3)  # 停顿3秒，确保机械臂完成动作


# Define head shake function at 50Hz
def head_shake():
    print('左右摆头')  # 开始左右摆头
    set_servo_pulses([1500, 1200, 550, 1200, 1610, 1400])  # 中间位置
    time.sleep(1)  # 停留1秒

    # 左右摆动两次
    for _ in range(2):
        set_servo_pulses([1100, 1200, 550, 1200, 1610, 1400])  # 左侧位置
        time.sleep(0.5)  # 0.5秒的停留时间

        set_servo_pulses([1900, 1200, 550, 1200, 1610, 1400])  # 右侧位置
        time.sleep(0.5)  # 0.5秒的停留时间

    set_servo_pulses([1500, 1200, 550, 1200, 1610, 1400])  # 回到中间位置
    time.sleep(1)  # 停留1秒

if __name__ == '__main__':
    pwm = PCA9685(0x40, debug=False)
    pwm.setPWMFreq(50)  # 设置PWM频率为50Hz

    try:
        # 执行左右摆头
        head_shake()
    
    except KeyboardInterrupt:
        print("程序中断，伺服电机停止")



def head_dance():
    # 跳舞
    print('机械臂跳舞')
    set_servo_pulses([1300, 1200, 550, 1200, 1610, 1400])  # 中间位置
    time.sleep(1)
    set_servo_pulses([1700, 1200, 550, 1000, 1610, 2100])  # 向下
    time.sleep(0.5)
    set_servo_pulses([1300, 1200, 550, 1400, 1610, 1400])  # 向上
    time.sleep(0.5)
    set_servo_pulses([1400, 1200, 550, 1000, 1610, 2100])  # 向下
    time.sleep(0.5)
    set_servo_pulses([1700, 1200, 550, 1400, 1610, 1400])  # 向上
    time.sleep(0.5)
    set_servo_pulses([1800, 1200, 550, 1200, 1610, 1400])  # 回到中间位置
    time.sleep(1)



def head_nod():
    # 点头
    print('机械臂点头')
    set_servo_pulses([1500, 1200, 550, 1200, 1610, 1400])  # 中间位置
    time.sleep(1)
    set_servo_pulses([1500, 1200, 550, 1000, 1610, 2100])  # 向下
    time.sleep(0.5)
    set_servo_pulses([1500, 1200, 550, 1400, 1610, 1400])  # 向上
    time.sleep(0.5)
    set_servo_pulses([1500, 1200, 550, 1000, 1610, 2100])  # 向下
    time.sleep(0.5)
    set_servo_pulses([1500, 1200, 550, 1400, 1610, 1400])  # 向上
    time.sleep(0.5)
    set_servo_pulses([1500, 1200, 550, 1200, 1610, 1400])  # 回到中间位置
    time.sleep(1)



def move_to_coords(X=150, Y=-130, HEIGHT_SAFE=230):
    print('移动至指定坐标：X {} Y {}'.format(X, Y))
    # 这部分根据机械臂具体实现情况修改舵机角度来控制机械臂位置

def single_joint_move(joint_index, angle):
    print('关节 {} 旋转至 {} 度'.format(joint_index, angle))
    set_servo_angle(joint_index, angle)
    time.sleep(2)


def move_to_top_view(target_pwm_values=[1500, 1450, 1000, 600, 1610, 1400]):
    """
    控制机械臂移动至俯视姿态，并模拟舵机移动速度。
    target_pwm_values: 目标 PWM 值列表，例如 [1500, 1450, 1000, 600, 1610, 1400]
    """
    print('移动至俯视姿态')
    
    # 获取当前 PWM 值
    current_pwm_values = [pca.getServoPulse(i) for i in range(len(target_pwm_values))]

    # 逐步将 PWM 值从当前移动到目标值，步长为 10
    for step in range(0, 101, 10):  # 步长为 10，相当于速度较快
        for i, target_pwm in enumerate(target_pwm_values):
            # 计算每步的 PWM 值
            current_pwm = current_pwm_values[i] + (target_pwm - current_pwm_values[i]) * (step / 100.0)
            pca.setServoPulse(i, int(current_pwm))  # 设置当前步的 PWM 值
            print(f'设置通道 {i} 的 PWM 脉宽为 {int(current_pwm)}')
        
        # 控制每一步之间的延时，使移动平滑
        time.sleep(0.005)  # 较短的延时适合快速移动

    # 最后停顿以确保机械臂稳定
    time.sleep(1)  



def top_view_shot(check=False):
    """
    拍摄俯视图并保存图像。
    check：是否需要人工确认拍摄成功。
    """
    print('移动至俯视姿态')
    move_to_top_view()
    
    # 初始化摄像头
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("无法打开摄像头")
        return

    time.sleep(0.3)  # 稍作延时以确保摄像头启动成功
    success, img_bgr = cap.read()
    
    if success:
        # 保存图像
        cv2.imwrite('temp/vl_now.jpg', img_bgr)
        print('图像已保存至 temp/vl_now.jpg')

        # 显示图像
        cv2.imshow('HANG_vlm', img_bgr)
        
        # 若需要人工确认拍摄成功
        if check:
            print('请确认拍照成功，按c键继续，按q键退出')
            while True:
                key = cv2.waitKey(10) & 0xFF
                if key == ord('c'):  # 按 c 键继续
                    break
                elif key == ord('q'):  # 按 q 键退出
                    cv2.destroyAllWindows()
                    cap.release()
                    raise NameError('按 q 键退出')

        # 等待一段时间后自动关闭显示窗口
        cv2.waitKey(0)
    else:
        print("图像读取失败")

    # 释放摄像头资源并关闭窗口
    cap.release()
    cv2.destroyAllWindows()
    
   

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# 定义机械臂的连杆长度
l1 = 105  # 第一段连杆长度
l2 = 265  # 第二段连杆长度
l3 = 220  # 第三段连杆长度
l4 = 145   # 末端连杆长度

# 图像坐标转换到机械臂坐标的函数
def eye2hand(X_im=160, Y_im=120):
    '''
    输入目标点在图像中的像素坐标，转换为机械臂坐标
    '''
    cali_1_im = [0, 478]                       # 左下角，第一个标定点的像素坐标
    cali_1_mc = [-21.8, -197.4]                # 左下角，第一个标定点的机械臂坐标
    cali_2_im = [640, 0]                       # 右上角，第二个标定点的像素坐标
    cali_2_mc = [215, -59.1]                   # 右上角，第二个标定点的机械臂坐标
    
    X_cali_im = [cali_1_im[0], cali_2_im[0]]
    X_cali_mc = [cali_1_mc[0], cali_2_mc[0]]
    Y_cali_im = [cali_2_im[1], cali_1_im[1]]
    Y_cali_mc = [cali_2_mc[1], cali_1_mc[1]]

    X_mc = int(np.interp(X_im, X_cali_im, X_cali_mc))
    Y_mc = int(np.interp(Y_im, Y_cali_im, Y_cali_mc))

    return X_mc, Y_mc

# 逆运动学函数
def inverse_kinematics(x, y, z, beta, type="lower"):
    """
    给定目标点(x, y, z)和末端执行器的角度beta，计算各个关节的角度
    """
    beta = np.deg2rad(beta)  # 将beta转换为弧度
    theta_0 = np.arctan2(y, x)  # 计算基座旋转角度
    theta_3 = beta  # 初始末端执行器角度

    # 计算在XZ平面上的临时坐标
    x_tmp = x / np.cos(theta_0) - l4 * np.cos(theta_3)
    z_tmp = z - l1 - l4 * np.sin(theta_3)
    r = np.sqrt(x_tmp**2 + z_tmp**2)  # 调整后点到原点的距离

    # 检查是否在机械臂的可达范围内
    max_reach = l2 + l3
    min_reach = abs(l2 - l3)
    if r > max_reach or r < min_reach:
        print("目标位置超出机械臂的可达范围")
        return None

    # 使用余弦定理计算theta_2
    cos_theta2 = (r**2 - l2**2 - l3**2) / (2 * l2 * l3)
    cos_theta2 = np.clip(cos_theta2, -1, 1)  # 限制范围以避免无效值
    theta_2 = np.arccos(cos_theta2)
    
    if type == "upper":  # “上”或“下”手肘位置选择
        theta_2 = -theta_2

    epsilon = np.arctan2(l3 * np.sin(theta_2), l2 + l3 * np.cos(theta_2))
    phi = np.arctan2(z_tmp, x_tmp)
    theta_1 = phi - epsilon
    theta_3 = theta_3 - theta_2 - theta_1  # 调整末端执行器角度

    # 将结果转换为角度
    theta_0 = np.degrees(theta_0)
    theta_1 = np.degrees(theta_1)
    theta_2 = np.degrees(theta_2)
    theta_3 = np.degrees(theta_3)
    
    return theta_0, theta_1, theta_2, theta_3

# 控制机械臂移动到目标图像坐标位置
def move_to_target(X_im, Y_im, Z=20, beta=45):
    """
    给定目标图像坐标(X_im, Y_im)，以及Z轴和末端执行器角度beta，控制机械臂移动
    """
    # 调用eye2hand将图像坐标转换为机械臂的物理坐标
    X_mc, Y_mc = eye2hand(X_im, Y_im)
    
    # 使用逆运动学计算关节角度
    joint_angles = inverse_kinematics(X_mc, Y_mc, Z, beta)
    
    # 检查是否得到了有效的关节角度
    if joint_angles is not None:
        theta_0, theta_1, theta_2, theta_3 = joint_angles
        print("移动到目标位置的关节角度：")
        print(f"Theta 0: {theta_0:.2f}°")
        print(f"Theta 1: {theta_1:.2f}°")
        print(f"Theta 2: {theta_2:.2f}°")
        print(f"Theta 3: {theta_3:.2f}°")
        # 在这里可以加入实际的机械臂控制代码，例如将这些角度发送到控制系统
    else:
        print("无法到达指定的目标位置。")

# 示例：给定图像坐标，转换并控制机械臂移动
#X_im, Y_im = 160, 120  # 示例目标图像坐标
#move_to_target(X_im, Y_im)







# 夹具控制函数
def activate_gripper(state):
    """
    控制夹具的函数
    state: True 表示抓取，False 表示释放
    """
    if state:
        print("夹具激活，进行抓取操作")
        # 在这里加入夹具抓取的实际控制代码，例如通过 PWM 信号抓紧物体
    else:
        print("夹具释放，放下物体")
        # 在这里加入夹具释放的实际控制代码，例如通过 PWM 信号松开物体

# 抓取和移动物体的函数
def pump_move(XY_START=[230, -50], HEIGHT_START=50, XY_END=[100, 220], HEIGHT_END=100, HEIGHT_SAFE=220, beta=45):
    """
    用夹具，将物体从起点吸取移动至终点
    
    XY_START: 起点机械臂坐标 (x, y)
    HEIGHT_START: 起点高度，方块用90，药盒子用70
    XY_END: 终点机械臂坐标 (x, y)
    HEIGHT_END: 终点高度
    HEIGHT_SAFE: 搬运途中安全高度
    """
    # 1. 移动到起点上方的安全高度
    print("移动到起点上方的安全高度")
    move_to_target_with_pwm(XY_START[0], XY_START[1], HEIGHT_SAFE, beta)

    # 2. 下降到起点的抓取高度并抓取物体
    print("下降到起点的抓取高度并准备抓取物体")
    move_to_target_with_pwm(XY_START[0], XY_START[1], HEIGHT_START, beta)
    activate_gripper(True)  # 激活夹具/机械爪进行抓取

    # 3. 提升到安全高度
    print("抓取完成，提升到安全高度")
    move_to_target_with_pwm(XY_START[0], XY_START[1], HEIGHT_SAFE, beta)

    # 4. 移动到终点上方的安全高度
    print("移动到终点上方的安全高度")
    move_to_target_with_pwm(XY_END[0], XY_END[1], HEIGHT_SAFE, beta)

    # 5. 下降到终点的放置高度并释放物体
    print("下降到终点的放置高度并释放物体")
    move_to_target_with_pwm(XY_END[0], XY_END[1], HEIGHT_END, beta)
    activate_gripper(False)  # 释放夹具/机械爪放下物体

    # 6. 回到安全高度
    print("放置完成，回到安全高度")
    move_to_target_with_pwm(XY_END[0], XY_END[1], HEIGHT_SAFE, beta)

# 示例调用
pump_move(
    XY_START=[230, -50], HEIGHT_START=90, 
    XY_END=[100, 220], HEIGHT_END=100, HEIGHT_SAFE=220
)



ModuleNotFoundError: No module named 'fcntl'